In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords') # Corrected function name to download stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english')) # Corrected function name to get English stopwords

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold # Corrected typo in function name
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
import re



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import pandas as pd
df = pd.read_csv('/content/Large language models (2024) (1).csv', encoding='latin-1')

In [5]:
print(df)

                      Model Comapany   Arch Parameters Tokens       Ratio  \
0                   Olympus   Amazon    TBA       2000   40000      20:01   
1                     GPT-5   OpenAI    TBA       2000     TBA        TBA   
2                     GPT-6   OpenAI    TBA        TBA     TBA        TBA   
3    AuroraGPT (ScienceGPT)      ANL    TBA       1000     TBA        TBA   
4                    Grok-2      xAI    TBA        TBA     TBA        TBA   
..                      ...      ...    ...        ...     ...        ...   
337                 RoBERTa  Meta AI  Dense      0.355    2200    6,198:1   
338                   GPT-2   OpenAI  Dense        1.5      10       7:01   
339                    BERT   Google  Dense        0.3     137  457:01:00   
340                   GPT-1   OpenAI  Dense      0.117   0.003       1:01   
341                  ULMFiT  Fast.ai  Dense      0.034     0.1       3:01   

    ALScore                Training dataset Release Date  \
0      29.8    

In [6]:
df.describe()

,Model,Comapany,Arch,Parameters,Tokens,Ratio,ALScore,Training dataset,Release Date,Notes,Playground
count,342,342,342,327,255,249,252,340,342,313,249
unique,340,122,5,127,102,101,44,21,48,305,222
top,MPT,Google,Dense,7,2000,20:01,0.4,"Wikipedia, books, common crawl",24-Apr,Chatbot,https://chat.openai.com/
freq,2,25,310,29,36,24,22,257,23,3,5


In [7]:
df.dtypes

Model               object
Comapany            object
Arch                object
Parameters          object
Tokens              object
Ratio               object
ALScore             object
Training dataset    object
Release Date        object
Notes               object
Playground          object
dtype: object

In [8]:
df.head()

,Model,Comapany,Arch,Parameters,Tokens,Ratio,ALScore,Training dataset,Release Date,Notes,Playground
0,Olympus,Amazon,TBA,2000,40000,20:01,29.8,TBA,TBA,New related Titan details: '$65m training run....,https://lifearchitect.ai/olympus/
1,GPT-5,OpenAI,TBA,2000,TBA,TBA,TBA,TBA,TBA,Due 2024.,https://lifearchitect.ai/gpt-5/
2,GPT-6,OpenAI,TBA,TBA,TBA,TBA,TBA,TBA,TBA,Due 2025.,https://lifearchitect.ai/gpt-6/
3,AuroraGPT (ScienceGPT),ANL,TBA,1000,TBA,TBA,TBA,TBA,TBA,https://tpc.dev/2023/11/10/tpc-announced-with-...,https://www.hpcwire.com/2023/11/13/training-of...
4,Grok-2,xAI,TBA,TBA,TBA,TBA,TBA,TBA,TBA,Due 2025.,https://twitter.com/elonmusk/status/1773655245...


In [9]:
df.tail()

,Model,Comapany,Arch,Parameters,Tokens,Ratio,ALScore,Training dataset,Release Date,Notes,Playground
337,RoBERTa,Meta AI,Dense,0.355,2200,"6,198:1",0.1,"Wikipedia, books, common crawl",19-Jul,"calcs: ""In total, this batch size and number o...",Hugging Face
338,GPT-2,OpenAI,Dense,1.5,10,7:01,0,Reddit,19-Feb,Reddit outbound only,Hugging Face
339,BERT,Google,Dense,0.3,137,457:01:00,0,"Wikipedia, books",18-Oct,NaN,Hugging Face
340,GPT-1,OpenAI,Dense,0.117,0.003,1:01,0,Books,18-Jun,"Books only. ""We train for 100 epochs on miniba...",Hugging Face
341,ULMFiT,Fast.ai,Dense,0.034,0.1,3:01,0,Wikipedia,18-Jan,Aussie Prof Jeremy Howard: https://www.abc.net...,NaN


In [10]:
!pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define input text
input_text = "Once upon a time"

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate text
output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decode and print generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:", generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: Once upon a time, the world was a place of great beauty and great danger. The world of the gods was the place where the great gods were born, and where they were to live.

The world that was created was not the same as the one that is now. It was an endless, endless world. And the Gods were not born of nothing. They were created of a single, single thing. That was why the universe was so beautiful. Because the cosmos was made of two


In [12]:
import pandas as pd

# Try different encodings
df = pd.read_csv('/content/Large language models (2024) (1).csv', encoding='latin-1')
# or
# df = pd.read_csv('/content/Large language models (2024) (1).csv', encoding='utf-16')
# or
# df = pd.read_csv('/content/Large language models (2024) (1).csv', encoding='iso-8859-1')

# If you're unsure of the encoding, try 'chardet'
!pip install chardet
import chardet
with open('/content/Large language models (2024) (1).csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
print(result)
# Use the encoding suggested by 'chardet' in the pd.read_csv function

{'encoding': 'MacRoman', 'confidence': 0.708200092958401, 'language': ''}


In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [14]:
# Define input text
input_text = "Hello, how are you today?"

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate response
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decode and print the output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Model response:", response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Model response: Hello, how are you today?

I'm so happy to be here. I'm really happy.
 (laughs)
...
, I think I've got a lot of work to do. It's been a long time


In [15]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [16]:
!openai migrate

Retrieving Grit CLI metadata from https://api.keygen.sh/v1/accounts/custodian-dev/artifacts/marzano-linux-x64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/marzano-linux-x64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/gouda-linux-x64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/workflow_runner-linux-x64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/cli-linux-x64
Fetching release URL from: https://api.keygen.sh//v1/accounts/custodian-dev/artifacts/timekeeper-linux-x64



Analyzing Finding files                                                         
Processed 0 files and found 0 matches


In [17]:
import openai

def ask_question(question, context):
    # Set your OpenAI API key here
    openai.api_key = "YOUR_API_KEY"  # Replace with your actual key

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Or another suitable chat model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{context}\n{question}"}
        ],
        max_tokens=150,
    )

    return response['choices'][0]['message']['content'].strip()  # Extract the answer

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [19]:
input_text = "Your input text here."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate text
output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your input text here.

You can also use the following command to add a new line to the file:
. /etc/rc.local/share/file.d/
 (This will add the line "file" to /usr/local.conf.)
,
 and
(This command will also add "filename" and "name" as the default file names.)


In [20]:
# Example with GPT-2 tokenizer
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Define your training texts here
train_texts = ["This is the first training text.", "Here's another one."]

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as the padding token

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

In [21]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.7):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


In [22]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600,
        style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [23]:
# Main loop
print("Chatbot: Hello! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break

    chat_prompt = f'User: {user_input}\nChatbot:'
    response = get_chat_response(chat_prompt)
    print("Chatbot:", response)


Chatbot: Hello! How can I assist you today?
You: exit
Chatbot: Goodbye!


In [24]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display


context = [ {'role':'system', 'content':"""
               I want you to act as an interviewing agent, named Tom,
               for an AI services company.
               You are interviewing candidates, appearing in the interview.
               I want you to only ask questions as the interviewer related to AI.
               Ask one questions at a time.

"""} ]


In [25]:
def on_send_click():
    user_input = user_input_text.get("1.0", tk.END).strip()
    if user_input:  # <- This is where the error is reported
        bot_response = get_bot_response(user_input)
        chat_history_text.config(state=tk.NORMAL)
        chat_history_text.insert(tk.END, f"You: {user_input}\n")
        chat_history_text.insert(tk.END, f"Bot: {bot_response}\n\n")
        chat_history_text.config(state=tk.DISABLED)
        user_input_text.delete("1.0", tk.END)
    else:
        messagebox.showwarning("Warning", "Please enter a message.")


In [26]:
def on_send_click():
    user_input = user_input_text.get("1.0", tk.END).strip()
    if user_input:
        bot_response = get_bot_response(user_input)
        chat_history_text.config(state=tk.NORMAL)
        chat_history_text.insert(tk.END, f"You: {user_input}\n")
        chat_history_text.insert(tk.END, f"Bot: {bot_response}\n\n")
        chat_history_text.config(state=tk.DISABLED)
        user_input_text.delete("1.0", tk.END)
    else:
        messagebox.showwarning("Warning", "Please enter a message.")


In [27]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import openai

# Function to get response from OpenAI LLM
def get_bot_response(user_input):
    try:
        response = openai.Completion.create(
            model="text-davinci-002",
            prompt=user_input,
            max_tokens=50
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Function to handle button click
def on_send_click():
    user_input = user_input_text.get("1.0", tk.END).strip()
    if user_input:
        bot_response = get_bot_response(user_input)
        update_chat_history(f"You: {user_input}")
        update_chat_history(f"Bot: {bot_response}\n")
        user_input_text.delete("1.0", tk.END)
    else:
        messagebox.showwarning("Warning", "Please enter a message.")

# Function to update chat history
def update_chat_history(message):
    chat_history_text.config(state=tk.NORMAL)
    chat_history_text.insert(tk.END, message + "\n\n")
    chat_history_text.config(state=tk.DISABLED)
    chat_history_text.yview(tk.END)

# Background image setup (if needed)
# Replace with your actual image path
# background_image = tk.PhotoImage(file="background_image.png")
# background_label = tk.Label(root, image=background_image)
# background_label.place(relwidth=1, relheight=1)



In [28]:
!pip install openai
import tkinter as tk
from tkinter import scrolledtext, messagebox
import openai

# ... (rest of your code)

In [29]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import openai

# Function to get response from OpenAI LLM
def get_bot_response(user_input):
    try:
        response = openai.Completion.create(
            model="text-davinci-002",
            prompt=user_input,
            max_tokens=50
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Function to handle button click
def on_send_click():
    user_input = user_input_text.get("1.0", tk.END).strip()
    if user_input:
        bot_response = get_bot_response(user_input)
        update_chat_history(f"You: {user_input}")
        update_chat_history(f"Bot: {bot_response}\n")
        user_input_text.delete("1.0", tk.END)
    else:
        messagebox.showwarning("Warning", "Please enter a message.")

# Function to update chat history
def update_chat_history(message):
    chat_history_text.config(state=tk.NORMAL)
    chat_history_text.insert(tk.END, message + "\n\n")
    chat_history_text.config(state=tk.DISABLED)
    chat_history_text.yview(tk.END)

# Background image setup (if needed)
# Replace with your actual image path
# background_image = tk.PhotoImage(file="background_image.png")
# background_label = tk.Label(root, image=background_image)
# background_label.place(relwidth=1, relheight=1)



In [30]:
import panel as pn
import openai

pn.extension()

# Function to get response from OpenAI LLM
def get_bot_response(user_input):
    try:
        response = openai.Completion.create(
            model="text-davinci-002",
            prompt=user_input,
            max_tokens=50
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

# Initialize chat history
chat_history = pn.widgets.TextAreaInput(value='', width=600, height=300)

# Input field for user messages
user_input = pn.widgets.TextInput(placeholder='Enter your message here...')

# Button to send message
send_button = pn.widgets.Button(name='Send')

# Function to handle button click
def on_send_click(event):
    user_message = user_input.value
    if user_message:
        bot_response = get_bot_response(user_message)
        chat_history.value += f"You: {user_message}\n"
        chat_history.value += f"Bot: {bot_response}\n\n"
        user_input.value = ''  # Clear input field

send_button.on_click(on_send_click)

# Layout the components
pn.Column(
    chat_history,
    pn.Row(user_input, send_button)
).servable()  # Make the app viewable

Column
    [0] TextAreaInput(height=300, sizing_mode='fixed', width=600)
    [1] Row
        [0] TextInput(placeholder='Enter your message h...)
        [1] Button(name='Send')